In [ ]:
from ortools.sat.python import cp_model

In [ ]:
initial_squares = [
    (3, 12, 4), # A square of size 3 at (12, 4)
    (4, 11, 0), (4, 33, 0), (4, 33, 4),
    (5, 9, 19), (5, 9, 24), (5, 33, 8),
    (6, 9, 13),
    (7, 31, 38), (7, 38, 8), (7, 38, 15), (7, 38, 38),
    (8, 9, 29), (8, 9, 37), (8, 29, 30), (8, 37, 0), (8, 37, 22), (8, 37, 30),
    (9, 0, 9), (9, 0, 18), (9, 0, 27), (9, 0, 36), (9, 15, 0), (9, 24, 0), (9, 29, 13)
]

In [ ]:
n = 9
grid_size = 45
model = cp_model.CpModel()

square_counts = {k: k for k in range(1, n + 1)}
for size, _, _ in initial_squares:
    square_counts[size] -= 1

x_vars = []
y_vars = []
sizes = []
x_intervals = []
y_intervals = []

for size in range(1, n + 1):
    count = square_counts[size]
    for i in range(count):
        x = model.NewIntVar(0, grid_size - size, f'x_{size}_{i}')
        y = model.NewIntVar(0, grid_size - size, f'y_{size}_{i}')
        x_vars.append(x)
        y_vars.append(y)
        sizes.append(size)

        x_interval = model.NewIntervalVar(x, size, x + size, f'x_int_{size}_{i}')
        y_interval = model.NewIntervalVar(y, size, y + size, f'y_int_{size}_{i}')
        x_intervals.append(x_interval)
        y_intervals.append(y_interval)

for i, (size, fx, fy) in enumerate(initial_squares):
    fx_const = model.NewConstant(fx)
    fy_const = model.NewConstant(fy)
    x_intervals.append(model.NewIntervalVar(fx_const, size, fx + size, f'x_fixed_{i}'))
    y_intervals.append(model.NewIntervalVar(fy_const, size, fy + size, f'y_fixed_{i}'))

model.AddNoOverlap2D(x_intervals, y_intervals)

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
print(f"Solution was optimal? {status == cp_model.OPTIMAL}")

# The squares that were put down
new_squares = [(sizes[i], solver.Value(x_vars[i]), solver.Value(y_vars[i])) for i in range(len(sizes))]

# And the location of the 1 square..
one_square = [(x,y) for size,x,y in new_squares if size == 1][0]
print(f"{one_square=}") # one_square=(20,18)